In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nslkdd/KDDTest+.arff
/kaggle/input/nslkdd/KDDTest-21.arff
/kaggle/input/nslkdd/KDDTest1.jpg
/kaggle/input/nslkdd/KDDTrain+.txt
/kaggle/input/nslkdd/KDDTrain+_20Percent.txt
/kaggle/input/nslkdd/KDDTest-21.txt
/kaggle/input/nslkdd/KDDTest+.txt
/kaggle/input/nslkdd/KDDTrain+.arff
/kaggle/input/nslkdd/index.html
/kaggle/input/nslkdd/KDDTrain+_20Percent.arff
/kaggle/input/nslkdd/KDDTrain1.jpg
/kaggle/input/nslkdd/nsl-kdd/KDDTest+.arff
/kaggle/input/nslkdd/nsl-kdd/KDDTest-21.arff
/kaggle/input/nslkdd/nsl-kdd/KDDTest1.jpg
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+_20Percent.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTest-21.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTest+.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+.arff
/kaggle/input/nslkdd/nsl-kdd/index.html
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+_20Percent.arff
/kaggle/input/nslkdd/nsl-kdd/KDDTrain1.jpg
/kaggle/input/nsl-kdd-augmented/smote_augmented.csv


In [2]:
!pip install torch torchvision scikit-learn pandas numpy

In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

In [4]:
BATCH_SIZE = 512
EPOCHS = 30
LR = 1e-3
LAMBDA = 0.5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from torch.utils.data import Dataset, DataLoader
import torch

# ---------------------------
# Paths
# ---------------------------
local_path = "/kaggle/input/nslkdd/"
train_path = "/kaggle/input/nsl-kdd-augmented/"

# ---------------------------
# Load datasets
# ---------------------------
# Note: Use header=0 if your augmented CSV has headers, otherwise None
df_train = pd.read_csv(train_path + "smote_augmented.csv") 
df_test = pd.read_csv(local_path + "KDDTest+.txt", header=None)

# ---------------------------
# Assign columns to test data
# ---------------------------
# Note: The 'outcome' is typically index 41 and 'level' is index 42 in the raw txt
columns = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land',
    'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
    'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
    'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count',
    'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate',
    'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
    'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
    'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate',
    'outcome', 'level'
]
df_test.columns = columns

# ---------------------------
# Separate features and labels
# ---------------------------
target = "outcome"
cat_cols = ['protocol_type', 'service', 'flag']

# Ensure num_cols uses all available features except target and level
X_train_aug = df_train.drop(columns=['outcome', 'level'], errors='ignore')
y_train_aug = df_train['outcome']

X_test = df_test.drop(columns=['outcome', 'level'], errors='ignore')
y_test = df_test['outcome']

num_cols = [c for c in X_train_aug.columns if c not in cat_cols]

# ---------------------------
# Filter test labels that exist in training
# ---------------------------
# CRITICAL: We only keep test rows where the outcome exists in the augmented training set
# If SMOTE reduced classes, we must filter the test set to avoid LabelEncoder errors
train_labels = set(y_train_aug.unique())
mask = y_test.isin(train_labels)

if mask.sum() == 0:
    print("WARNING: No matching labels between train and test! Checking for string/int mismatch...")
    # Standardize to string in case of type mismatch
    y_train_aug = y_train_aug.astype(str)
    y_test = y_test.astype(str)
    train_labels = set(y_train_aug.unique())
    mask = y_test.isin(train_labels)

X_test = X_test[mask].reset_index(drop=True)
y_test = y_test[mask].reset_index(drop=True)

print(f"Test data retained: {len(X_test)} samples")
print("Filtered test label distribution:\n", y_test.value_counts())

# ---------------------------
# Column Transformer: scale numeric + encode categorical
# ---------------------------
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols)
    ]
)

X_train_processed = preprocessor.fit_transform(X_train_aug)
X_test_processed  = preprocessor.transform(X_test)

# ---------------------------
# Encode target labels
# ---------------------------
label_enc = LabelEncoder()
y_train_encoded = label_enc.fit_transform(y_train_aug)
y_test_encoded  = label_enc.transform(y_test)

print(f"Processed training shape: {X_train_processed.shape}")
print(f"Processed test shape: {X_test_processed.shape}")

class TabDataset(Dataset):
    def __init__(self, X_processed, y_encoded, cat_indices, num_indices):
        # X_processed is the 122-feature matrix from the ColumnTransformer
        self.x_cat = torch.tensor(X_processed[:, cat_indices], dtype=torch.long)
        self.x_num = torch.tensor(X_processed[:, num_indices], dtype=torch.float32)
        self.y = torch.tensor(y_encoded, dtype=torch.long)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.x_cat[idx], self.x_num[idx], self.y[idx]

# Identify indices for cat and num features in the 122-column processed matrix
# Assuming 'preprocessor' is the ColumnTransformer from previous step
cat_slice = preprocessor.output_indices_['cat']
num_slice = preprocessor.output_indices_['num']

train_dataset = TabDataset(X_train_processed, y_train_encoded, cat_slice, num_slice)
test_dataset = TabDataset(X_test_processed, y_test_encoded, cat_slice, num_slice)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Test data retained: 18794 samples
Filtered test label distribution:
 outcome
normal             9711
neptune            4657
guess_passwd       1231
warezmaster         944
satan               735
smurf               665
back                359
portsweep           157
ipsweep             141
nmap                 73
pod                  41
buffer_overflow      20
multihop             18
rootkit              13
teardrop             12
land                  7
ftp_write             3
perl                  2
loadmodule            2
phf                   2
imap                  1
Name: count, dtype: int64
Processed training shape: (557934, 122)
Processed test shape: (18794, 122)


In [21]:
# 1. Determine number of classes from the LabelEncoder
num_classes = len(label_enc.classes_)

# 2. Identify column indices for cat and num features in the processed matrix
# categorical_cols were ['protocol_type', 'service', 'flag']
cat_cols = ['protocol_type', 'service', 'flag']
num_cols = [c for c in X_train_aug.columns if c not in cat_cols]

# We need to know which columns in X_train_processed belong to 'num' and 'cat'
# Based on your ColumnTransformer: num is first, then cat
num_feature_count = len(num_cols)
# Categorical features are One-Hot Encoded, so we find where they start
# X_train_processed[:, :num_feature_count] is numerical
# X_train_processed[:, num_feature_count:] is categorical (as one-hot)

class TabDataset(Dataset):
    def __init__(self, X, y, num_feat_count):
        # Convert sparse to dense if necessary
        if hasattr(X, "toarray"): X = X.toarray()
        
        # In your ColumnTransformer, numerical columns come first
        self.x_num = torch.tensor(X[:, :num_feat_count], dtype=torch.float32)
        
        # For TabTransformer, categorical features usually need Label Encoding 
        # for embeddings, but since you used OneHotEncoder in the preprocessor,
        # we will pass the numerical and encoded features as separate tensors.
        # NOTE: To use nn.Embedding, we need the raw categorical integer codes.
        self.x_cat = torch.tensor(X[:, num_feat_count:], dtype=torch.float32) 
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.x_cat[idx], self.x_num[idx], self.y[idx]

BATCH_SIZE = 512
train_dataset = TabDataset(X_train_processed, y_train_encoded, num_feature_count)
test_dataset  = TabDataset(X_test_processed, y_test_encoded, num_feature_count)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [22]:
print(df_train.columns.tolist())
print(df_test.columns.tolist())

['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'level', 'outcome']
['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'n

In [23]:
import torch.nn as nn

class TabTransformer(nn.Module):
    def __init__(self, cat_feature_count, num_features, num_classes, emb_dim=32, depth=4, heads=4, dropout=0.1):
        super().__init__()

        # Since your preprocessor used OneHotEncoder, cat_feature_count is the 
        # total number of binary columns created (e.g., ~84).
        # We project these into the embedding space to work with the Transformer.
        self.cat_projection = nn.Linear(cat_feature_count, emb_dim)

        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=emb_dim,
                nhead=heads,
                dim_feedforward=emb_dim * 4,
                dropout=dropout,
                batch_first=True
            ),
            num_layers=depth
        )

        self.mlp = nn.Sequential(
            nn.Linear(emb_dim + num_features, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x_cat, x_num):
        # x_cat is [batch, cat_onehot_dim]
        x = self.cat_projection(x_cat) # [batch, emb_dim]
        x = x.unsqueeze(1)             # [batch, 1, emb_dim] for Transformer
        x = self.transformer(x)
        x = x.squeeze(1)               # [batch, emb_dim]
        
        x = torch.cat([x, x_num], dim=1)
        return self.mlp(x)

# Initialization
cat_onehot_dim = X_train_processed.shape[1] - num_feature_count
model = TabTransformer(cat_onehot_dim, num_feature_count, num_classes).to(DEVICE)

if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)

optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()

In [25]:
import torch
import torch.nn as nn

# 1. Define the FocalLoss class
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2):
        super().__init__()
        self.gamma = gamma
        self.alpha = alpha # If provided, this should be a tensor of weights per class

    def forward(self, inputs, targets):
        # inputs: [batch_size, num_classes], targets: [batch_size]
        ce_loss = nn.functional.cross_entropy(inputs, targets, reduction='none', weight=self.alpha)
        pt = torch.exp(-ce_loss)
        loss = ((1 - pt) ** self.gamma * ce_loss).mean()
        return loss

# 2. Re-initialize the model
cat_onehot_dim = X_train_processed.shape[1] - num_feature_count
model = TabTransformer(cat_onehot_dim, num_feature_count, num_classes).to(DEVICE)

if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)

# 3. Setup Optimizer and Focal Loss
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

# OPTIONAL: Calculate alpha weights to further handle imbalance
# This gives more weight to classes with fewer samples
class_counts = np.bincount(y_train_encoded)
weights = 1.0 / (class_counts + 1e-6)
weights = torch.tensor(weights / weights.sum() * num_classes).float().to(DEVICE)

# Set criterion to FocalLoss
criterion = FocalLoss(alpha=weights, gamma=2)

# ---------------------------
# Training Loop remains the same
# ---------------------------
def train_epoch(model, loader):
    model.train()
    total_loss = 0
    for x_cat, x_num, y in loader:
        x_cat, x_num, y = x_cat.to(DEVICE), x_num.to(DEVICE), y.to(DEVICE)
        
        logits = model(x_cat, x_num)
        
        # Consistency Regularization
        noise = torch.randn_like(x_num) * 0.05
        logits_aug = model(x_cat, x_num + noise)
        
        loss_cls = criterion(logits, y)  # Now uses Focal Loss
        loss_cons = torch.mean((logits - logits_aug)**2)
        loss = loss_cls + LAMBDA * loss_cons

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

In [26]:
from sklearn.metrics import accuracy_score, classification_report

def train_epoch(model, loader):
    model.train()
    total_loss = 0
    for x_cat, x_num, y in loader:
        x_cat, x_num, y = x_cat.to(DEVICE), x_num.to(DEVICE), y.to(DEVICE)
        
        logits = model(x_cat, x_num)
        
        # Consistency Regularization (Optional, based on your previous code)
        noise = torch.randn_like(x_num) * 0.05
        logits_aug = model(x_cat, x_num + noise)
        
        loss_cls = criterion(logits, y)
        loss_cons = torch.mean((logits - logits_aug)**2)
        loss = loss_cls + LAMBDA * loss_cons

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def evaluate(model, loader):
    model.eval()
    all_preds, all_true = [], []
    with torch.no_grad():
        for x_cat, x_num, y in loader:
            x_cat, x_num = x_cat.to(DEVICE), x_num.to(DEVICE)
            logits = model(x_cat, x_num)
            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_true.extend(y.numpy())

    print("Accuracy:", accuracy_score(all_true, all_preds))
    print(classification_report(all_true, all_preds, target_names=label_enc.classes_))

# Execution Loop
for epoch in range(EPOCHS):
    loss = train_epoch(model, train_loader)
    print(f"Epoch {epoch+1}: Loss = {loss:.4f}")

evaluate(model, test_loader)

Epoch 1: Loss = 0.2281
Epoch 2: Loss = 0.1563
Epoch 3: Loss = 0.1480
Epoch 4: Loss = 0.1438
Epoch 5: Loss = 0.1415
Epoch 6: Loss = 0.1399
Epoch 7: Loss = 0.1389
Epoch 8: Loss = 0.1382
Epoch 9: Loss = 0.1372
Epoch 10: Loss = 0.1374
Epoch 11: Loss = 0.1365
Epoch 12: Loss = 0.1358
Epoch 13: Loss = 0.1353
Epoch 14: Loss = 0.1352
Epoch 15: Loss = 0.1350
Epoch 16: Loss = 0.1347
Epoch 17: Loss = 0.1346
Epoch 18: Loss = 0.1343
Epoch 19: Loss = 0.1338
Epoch 20: Loss = 0.1337
Epoch 21: Loss = 0.1335
Epoch 22: Loss = 0.1337
Epoch 23: Loss = 0.1338
Epoch 24: Loss = 0.1333
Epoch 25: Loss = 0.1327
Epoch 26: Loss = 0.1332
Epoch 27: Loss = 0.1328
Epoch 28: Loss = 0.1328
Epoch 29: Loss = 0.1326
Epoch 30: Loss = 0.1327
Accuracy: 0.8183994891986804
                 precision    recall  f1-score   support

           back       0.98      1.00      0.99       359
buffer_overflow       0.04      0.20      0.06        20
      ftp_write       0.03      0.33      0.06         3
   guess_passwd       0.38     

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_